# LLM Agent를 만드는 여정 (5) Chain Of Density 기법

## 오늘의 목표 : CoD(Chain of Density)를 활용하여 장문 요약하기

- [From Sparse to Dense: GPT-4 Summarization with Chain of Density Prompting](https://arxiv.org/pdf/2309.04269)

#### Setup

In [2]:
import dotenv
import os
dotenv.load_dotenv("../.env")
if not "OPENAI_API_KEY" in os.environ:
    raise Exception("OPENAI_API_KEY가 환경변수에 존재하지 않습니다.")

#### 모델 불러오기

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

## CoD 프롬프트 구성하기

In [35]:
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template('''Article: {article}

You will generate increasingly concise, entity-dense summaries of the above article.

Repeat the following 2 steps {iteration} times.

Step 1. Identify 1-3 informative entities (";" delimited) from the article that are missing from the previously generated summary.

Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is:

- Relevant to the main story,

- Specific yet concise (5 words or fewer),

- Novel (not in the previous summary),

- Faithful (present in the article),

- Anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long ({num_sentences} sentences, ~{num_words} words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~{num_words} words.

- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.

- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".

- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.

- Missing entities can appear anywhere in the new summary.

- Use Korean.

- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary.''')

prompt = ChatPromptTemplate(messages=[human_template], input_variables=['article', 'iteration', 'num_sentences','num_words'])

### Langchain으로 구성하기

In [36]:
from pydantic import BaseModel, Field
from typing import List

class SummaryOutput(BaseModel):
    missing_entities: str
    denser_summary: str

class CoDOutput(BaseModel):
    summaries: List[SummaryOutput]

In [37]:
cod_llm = llm.with_structured_output(CoDOutput)

summary_chain = prompt | cod_llm

### 오건영님의 페이스북 에세이 요약하기

오건영 부부장님은 매크로 금융 시장에 대해 질 좋은 에세이를 페이스북에 자주 작성하는데, 쉽게 읽기에는 분량이 꽤 많은 편에 속합니다. 이를 간단하게 요약해 보도록 하겠습니다.

In [53]:
article = ("이번 주 내내 아침 일정이 있어서 제대로 에세이를 올리지 못했네요. 늦었지만 금요일에 작성해봅니다. 중요한 이슈들이 많았는데요, 주말 에세이에서 상당 부분 커버하도록 하겠습니다. 😭"
"우선 자산 시장 분위기가 나쁘지 않습니다. 미국의 50bp 금리 인하가 꽤 큰 나비 효과를 주고 있죠. 일단 깜짝 0.5%금리 인하에 일본은 기존의 금리 인상 계획을 미루는 듯한 분위기죠. 우에다 총재는 시간을 벌었다는 식으로 코멘트를 했습니다."
"미국의 50bp 인하는 미국 25bp 인하 & 일본도 25bp 인상 조합과 동일하다고 볼 수 있겠죠. 그런데 미국이 빅스텝으로 나왔으니 일본은 나서기가 무색해진 겁니다. 그래서 우에다 총재는 시간을 벌었다는 얘기와 함께 엔화 역시 162엔에서 142엔까지 빠른 강세를 보여준 만큼…" 
"엔 약세 지속에 대한 우려가 줄었으니.. 천천히 가도 될 것 같다는 얘기를 한 거겠죠. 그런데 이런 얘기도 너무 대놓고 하면 안됩니다. 시장이 그 패를 읽어버리거든요.. 그랬더니 엔화가 바로 달러 당 145엔 수준으로 약세를 보이게 되죠. 시장이 참 빨라졌습니다. "
"예전에는 이런 코멘트의 행간을 읽으려는 시도가 많았는데요.. 이제는 그냥 순식간에 엔 약세로 대응해버리네요.."
"중국은 지준율 50bp 인하에 역레포 금리 인하, 주택금융에 대한 완화 조처, 그리고 후속으로 재정 지출을 통한 소비 확대 정책까지 연달아 내놓고 있습니다. 중국 입장에서는 두가지 고민을 해야 합니다. 하나는 중국의 부양책이 자칫해서 자산 가격 버블, 특히 부동산에 대한 버블을 자극할 가능성이죠." 
"생산성이 높지 않은 부동산 시장으로 돈이 너무 많이 몰리게 되면 중국의 중장기 성장을 담보하기 어려워집니다. 이게 과거 일본의 패착이었던 만큼 중국은 그 길을 가고 싶지 않죠. 그래서 경기 둔화 우려가 확연해지고 있음에도 부양에 나서지 않은 면이 강합니다. "
"하나 더.. 미국의 금리가 높고 미국의 성장이 강한 상황에서 섣불리 금리 인하에 나섰다가… 강한 부양에 대한 의지를 표명했다가 되려 자본 유출 압력 증가라는 상황에 처할 수 있죠. 강한 부양이 자본 유출 압력에 놓이게 되면 중국은 그야말로 외통에 걸리게 될 겁니다. "
"성장을 부양하려면 돈을 뿌려야 하는데.. 돈을 뿌리면 자본 유출로 흘러나가니.. 이렇게 되면 중국 답없다.. 라는 확신과 함께 경제 주체들과 투자자들의 실망이 커질 수 있죠."
"그래서 어려운 상황임에도 적극 완화보다는 소극적인, 간헐적인 완화로 대응하고 있었는데요.. 이번에 미국이 금리 인하에 나서게 되고.. 이후 빠른 인하 가능성을 시사하자 중국 역시 자본 유출 압력을 낮추면서 과감한 부양에 나설 수 있게 된 것이죠." 
"중국 자본 유출 우려가 상당히 컸던 2016년 2월 말… 미국은 금리 인상을 늦춰주었구요… 중국은 이에 발맞춰 지준율을 인하하면서 국제 공조에 나선 바 있습니다." 
"중국의 부양책이 효과가 있는지는 위안화 환율을 보시면 답이 나옵니다. 금리가 내려간다는 것은 위안화 공급이 늘어난다는 의미인데요.. 위안화 공급은 위안화 약세를 가리킵니다. 그런데.. 그런 위안화 공급이 중국의 성장을 자극한다면…. "
"위안화 공급으로 중국의 성장이 만들어지면서 해외에서 대중 투자 심리가 강해질 수 있죠. 이는 중국으로의 자본 유입을 촉발, 되려 위안화 강세로 이어질 수 있습니다. 최근 위안화가 달러 당 7위안 초반까지 밀려내려오면서 훈훈한 분위기를 연출하고 있죠. "
"중국 부양책에 대한 기대가 중국의 성장 제고 및 중국의 금융 시장 개선에도 긍정적 영향을 주는 듯 합니다. 중국과 미국의 경제 실무진이 지속 대화를 나누었다고 하는데요.. 이게 국제 공조에도 영향을 준 것 아닌가.. 하는 생각도 조심스레 해봅니다."
"스위스는 얘기가 조금 다릅니다만 금리 인하에 나섰죠. 미국의 빠른 금리 인하가 달러 약세를 촉발하게 됩니다. 달러 약세는 스위스 프랑에는 강세 압력으로 작용하죠. 소규모 개방 국가 중 하나라고 할 수 있는 스위스 역시 수출 성장에 의존하는 면이 강합니다."
"미국 달러 약세 & 스위스 프랑의 과도한 절상 우려.. 이 둘이 맞붙게 되자 스위스 중앙은행은 환율에 초점을 맞추면서 25bp 금리 인하에 나서게 됩니다. 올해 들어 세번째 인하인데요… 시장에서는 스위스 프랑 강세를 제어하려면 25bp가 아니라 50bp인하가 필요하다…라는 전망이 있었던 만큼… "
"이번 25bp인하는 시장의 기대를 충족시켜주지는 못했죠. 스위스 프랑 강세가 이어지고 있는데요… 이는 연내 스위스의 추가 인하 가능성을 높이는 요인입니다. 그리고 스위스의 금리 인하는 돌고 돌아 다시금 유로존의 금리 인하를 자극하게 되겠죠. "
"유로존이 조금 흥미로운데요… 최근 보면 이탈리아나 스페인, 그리고 그리스 등의 성장세가 양호합니다. 유로존 힘들다는 얘기 많이 들었는데.. 무신 소리냐.. 라고 하실 수도 있겠지만 관광객이 너무 많이 몰려와서 숙박할 때 숙박세를 따로 걷겠다나.. "
"이런 얘기들이 나오고 있죠. 대신 러시아에서 저렴한 에너지를 가져와서 중국에 수출을 하면서 강한 대외 수출 성장을 했던 독일 경제는 상당한 어려움에 빠져있습니다. 굳이 이유를 말씀드리지 않아도 22년 2월 24일 시작된 러-우 전쟁을 생각하면… 독일이 힘든 이유가 설명이 되겠죠. "
"독일이 어렵고 그리스, 스페인, 이탈리아 등의 성장 모멘텀이 양호합니다. 유로존은 연합체이다 보니.. 독일의 침체에 가까운 성장세에도 전체를 보면 내년도 유로존 성장에 대한 기대가 크죠. 미국의 성장세는 올해보다 내년에 약해질 것으로 보입니다(그래서 금리를 인하한 거겠죠). "
"반대로 유로존의 성장은 강해질 것이라는 기대가 있죠. 미국의 성장 둔화 & 유로존 성장 기대… 이 둘의 조합은 달러 약세 & 유로화 강세를 만들게 될 겁니다. 유로 강세는 독일 수출에도 힘겨운 일이겠지만 관광 수입 등을 생각해도 달가운 얘기는 아니겠죠. "
"스위스 만큼 환율에 목을 메지는 않아도 유로존 역시 금리 인하를 추가로 고민할 수 밖에 없는 상황으로 몰리게 될 것으로 보입니다."
"국제 공조까지 나왔으니.. 통화 완화 훈풍에 모든 건 잘될거야.. 라는 기대가 올라오는 듯 합니다. 그 반대편의 이야기를 주말에 이어가겠습니다. 감사합니다.")

In [57]:
outputs = summary_chain.invoke({
    "article" : article, 
    "iteration": 5,
    'num_sentences': 5,
    "num_words": 120,
})

In [58]:
for i, output in enumerate(outputs.summaries,1):
    print(f"{i}차 요약")
    print(output)
    print("----\n")

1차 요약
missing_entities='미국 50bp 금리 인하; 일본 금리 인상 계획 연기; 중국 지준율 50bp 인하' denser_summary='이번 주 자산 시장은 미국의 50bp 금리 인하로 인해 긍정적인 분위기입니다. 일본은 금리 인상 계획을 연기하며, 우에다 총재는 시간을 벌었다고 언급했습니다. 엔화는 강세를 보였으나 다시 약세로 돌아섰습니다. 중국은 50bp 지준율 인하와 주택금융 완화 조처를 통해 자산 시장을 부양하고 있습니다. 스위스는 25bp 금리 인하를 단행했으며, 프랑 강세로 추가 인하 가능성이 점쳐지고 있습니다.'
----

2차 요약
missing_entities='스위스 프랑 강세; 유로존 성장 기대; 미국 성장 둔화' denser_summary='미국의 50bp 금리 인하로 글로벌 자산 시장에 긍정적인 영향이 미치고 있습니다. 일본은 금리 인상 계획을 연기하며 엔화는 일시적 강세 후 약세로 돌아섰습니다. 중국은 50bp 지준율 인하와 주택금융 완화로 자산 시장을 부양하고 있습니다. 스위스는 프랑 강세를 제어하기 위해 25bp 금리 인하를 단행했으며, 추가 인하 가능성이 있습니다. 유로존은 성장 기대가 커지고 있으며, 미국은 성장 둔화가 예상됩니다.'
----

3차 요약
missing_entities='독일 경제 어려움; 국제 공조; 러-우 전쟁' denser_summary='미국 50bp 금리 인하로 글로벌 자산 시장에 긍정적 영향이 있습니다. 일본은 금리 인상 계획을 연기하며 엔화는 일시적 강세 후 약세로 돌아섰습니다. 중국은 50bp 지준율 인하와 주택금융 완화로 자산 시장을 부양하고 있습니다. 스위스는 프랑 강세를 제어하기 위해 25bp 금리 인하를 단행했으며 추가 인하가 예상됩니다. 유로존은 성장 기대가 있으나 독일 경제는 러-우 전쟁으로 어려움을 겪고 있습니다. 국제 공조가 이루어지고 있으며, 미국은 성장 둔화가 예상됩니다.'
----

4차 요약
missing_entities='유로화 강세; 엔화 약세; 스위스 